In [1]:
import cv2
import cv2 as cv
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget


ImportError: cannot import name 'jpeg_to_bgr8'

In [11]:
# open camera
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

w,h = 640,480
camera = CSICamera(width=w, height=h)
# camera = USBCamera(width=224, height=224)

camera.running = True

In [12]:
# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

In [13]:
# set distort matrixs
dist = np.float32([[-0.31085325, -0.11558236,  0.00249467, -0.00088277,  0.51442531]])
mtx = np.float32(
    [[402.60350228,   0.,         263.30000918],
 [  0.,         537.76023089, 278.24728515],
 [  0.,           0.,           1.,        ]]
)

In [14]:
#Get the roi for projection
import numpy as np
from PIL import Image
from io import BytesIO
camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)


newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
print(roi)

# undistort image
distorted_widget = ClickableImageWidget(width=camera.width, height=camera.height)
corners_widget = ipywidgets.Image(width=camera.width, height=camera.height)

def distort_image(change):
    global mtx, dist, newcameramtx
    frame = change['new']
    distorted = cv.undistort(frame, mtx, dist, None, newcameramtx)
    
    img = Image.fromarray(distorted)
    with BytesIO() as f:
        img.save(f, format='JPEG')
        distorted_widget.value = f.getvalue()

def get_project_roi(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # display saved snapshot
        frame = jpeg_to_bgr8(distorted_widget.value)

        frame = cv2.circle(frame, (x, y), 8, (0, 255, 0), 3)
        corners_widget.value = bgr8_to_jpeg(frame)
        
camera.observe(distort_image, names='value')

distorted_widget.on_msg(get_project_roi)

data_collection_widget = ipywidgets.VBox([
    camera_widget, distorted_widget, corners_widget
])
display(data_collection_widget)

(12, 146, 456, 255)


In [8]:
camera.unobserve_all()

In [2]:
camera.running = False  # Stop the capture loop
camera.cap.release()    # Release the underlying VideoCapture object
del camera              # Delete the camera instance to free memory

import gc
gc.collect()

NameError: name 'camera' is not defined